# Исследование надежности заемщиков

**Описание проекта:**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


In [1]:
#импорт библиотеки
import pandas as pd

## Загрузка данных

In [2]:
#загрузка данных 
try:
    data = pd.read_csv('/.../data.csv')
except:
    data = pd.read_csv('https://.../data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Данные загружены, в days_employed и total_income 2174 пропуска. 

## Предобработка данных

### Удаление пропусков

Заполним пропуски в `total_income` медианным значением по каждому типу занятости `income_type`

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [8]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

In [9]:
#заменим все отрицательные значения положительными
data['days_employed'] = data['days_employed'].abs()

In [10]:
#для каждого типа занятости посчитаем медианное значение трудового стажа `days_employed` в днях
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Возможно, проблема в том, что данные в этих категориях записаны в часах. Оставим данные как есть, так как этот столбец не понадобится нам для исследования

In [11]:
#выведем перечень уникальных значений столбца `children`
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [12]:
#удалим аномальные значения
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [14]:
#заменим на целочисленный тип столбец total_income
data['total_income'] = data['total_income'].astype(int)

In [15]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [16]:
#приведем к нижнему регистру столбец education
data['education'] = data['education'].str.lower()

In [17]:
#удалим дубликаты
data = data.drop_duplicates()

In [18]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [19]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Сформируем новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [21]:
#выведем на экран перечень уникальных целей взятия кредита из столбца purpose.
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [22]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [24]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

## Исследовательский анализ данных

 - Посмотрим, есть ли зависимость между количеством детей и возвратом кредита в срок

In [25]:
result_1 = data.groupby('children')['debt'].agg(mean=('mean'), sum=('sum'), count=len)
result_1['mean'] = (result_1['mean'] * 100).round(2)
result_1

,mean,sum,count
children,,,
0,7.54,1063,14091
1,9.23,444,4808
2,9.45,194,2052
3,8.18,27,330
4,9.76,4,41
5,0.00,0,9


**Вывод:**  Зависимость между количеством детей и возвратом кредита в срок есть, так как меньшая доля должников находится в группе людей, не имеющих детей (за исключением людей с 5 детьми, у которых нет задолженности по кредиту, но эту категорию можно не учитывать, так как она самая малочисленная, и у людей с 4 детьми - самая высокая доля задолженности).

 - Посмотрим, есть ли зависимость между семейным положением и возвратом кредита в срок

In [26]:
result_2 = data.groupby('family_status')['debt'].agg(mean=('mean'), sum=('sum'), count=len)
result_2['mean'] = (result_2['mean'] * 100).round(2)
result_2

,mean,sum,count
family_status,,,
Не женат / не замужем,9.76,273,2796
в разводе,7.06,84,1189
вдовец / вдова,6.62,63,951
гражданский брак,9.31,385,4134
женат / замужем,7.56,927,12261


Вывод: Исходя из того, что доля должников в двух противоположных категориях "в разводе" и "женат/не замужем" примерно одинакова, можно сделать вывод, что семейное положение в меньшей степени влияет на погашение кредита в срок, но если учесть другие категории, то состоящие или состоявшие в браке более финансово дисциплинированы

 - Посмотрим, есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [27]:
result_3 = data.groupby('total_income_category')['debt'].agg(mean=('mean'), sum=('sum'), count=len)
result_3['mean'] = (result_3['mean'] * 100).round(2)
result_3

,mean,sum,count
total_income_category,,,
A,8.00,2,25
B,7.06,354,5014
C,8.50,1353,15921
D,6.02,21,349
E,9.09,2,22


**Вывод:** Категории A и Е оказывают незначительное влияние на зависимость, поскольку количество людей с таким уровнем дохода наименьшее. Люди среднего достатка из категории С имеют высокий процент задолженности, при этом категория наиболее многочисленна. Небольшая группа людей с доходом ниже среднего из категрии D имеет процент задолженности самый низкий. Зависимость отсутствует и, скорее всего, другие факторы, такие как кредитная история и стабильность работы, могут оказывать более значительное влияние, сравнение категорий B и С позволяет предположить, что более состоятельный клиент - более надежный.

 - Посмотрим, как разные цели кредита влияют на его возврат в срок

In [28]:
result_4 = data.groupby('purpose_category')['debt'].agg(mean=('mean'), sum=('sum'), count=len)
result_4['mean'] = (result_4['mean'] * 100).round(2)
result_4

,mean,sum,count
purpose_category,,,
операции с автомобилем,9.35,400,4279
операции с недвижимостью,7.26,780,10751
получение образования,9.25,369,3988
проведение свадьбы,7.91,183,2313


**Вывод:** Самая обширная категория - операции с недвижимостью - имеет самый низкий процент должников, так же как и самая малочисленная категория - проведение свадьбы. Из этого можно сделать вывод, что люди, берущие кредиты на эти цели, с большей вероятностью возвращают кредит в срок, чем люди из категорий, связанных с автомобилями и образованием.

### Шаг 4: общий вывод.

В ходе проекта была выполнена предобработка данных, включающая удаление или заполнение пропусков, обработку аномальных значений и удаление дубликатов. Также была выполнена категоризация некоторых данных для обеспечения их удобства в последующем анализе. В результате анализа были сделаны следующие выводы: количество детей оказывает влияние на своевременное погашение кредита, в то время как семейное положение или уровень дохода не оказывают существенного влияния. Было выявлено, что люди, которые брали кредит на операции с недвижимостью или проведение свадьбы, с большей вероятностью погашают его в срок.

Плюс можно дать рекомендацию заказчику по сбору данных:

  - в анкете сделать поля о доходе и стаже обязательными к заполнению, добавить автопроверку на формат вводимых пользователем данных;
  - изменить шкалу градации заемщиков по доходам, для получения более сбалансированных выборок